In [1]:
import pandas as pd
from utils import MysqlConnector
conn = MysqlConnector()

In [4]:
conn.execute("USE dw;")
conn.execute("""
ALTER TABLE customer_dim
  ADD shipping_address CHAR(50) AFTER customer_state
, ADD shipping_zip_code INT(5) AFTER shipping_address
, ADD shipping_city CHAR(30) AFTER shipping_zip_code
, ADD shipping_state CHAR(2) AFTER shipping_city
;
""")
conn.execute("""
ALTER TABLE customer_stg
  ADD shipping_address CHAR(50) AFTER customer_state
, ADD shipping_zip_code INT(5) AFTER shipping_address
, ADD shipping_city CHAR(30) AFTER shipping_zip_code
, ADD shipping_state CHAR(2) AFTER shipping_city
;
""")
conn.execute("""
ALTER TABLE sales_order_fact
  ADD order_quantity INT AFTER order_amount
;
""")


In [5]:
conn.execute("""USE source;""")
conn.execute("""DROP TABLE sales_order;""")
conn.execute("""CREATE TABLE sales_order
( order_number INT
, customer_number INT
, product_code INT
, order_date DATE
, entry_date DATE
, order_amount DECIMAL (10, 2)
, order_quantity INT)
;""")
conn.execute("""INSERT INTO sales_order VALUES
  (42, 1, 1, '2007-03-02', '2007-03-02', 1000, 10)
, (43, 2, 2, '2007-03-02', '2007-03-02', 2000, 20)
, (44, 3, 3, '2007-03-02', '2007-03-02', 4000, 40)
, (45, 4, 4, '2007-03-02', '2007-03-02', 6000, 60)
, (46, 5, 7, '2007-03-02', '2007-03-02', 2500, 25)
, (47, 6, 7, '2007-03-02', '2007-03-02', 5000, 50)
, (48, 7, 8, '2007-03-02', '2007-03-02', 9500, 75)
, (49, 8, 9, '2007-03-02', '2007-03-02', 12000, 10)
, (50, 9, 9, '2007-03-02', '2007-03-02', 12000, 10)
, (51, 10, 2, '2007-03-02', '2007-03-02', 2400, 10)
, (52, 10, 3, '2007-03-02', '2007-03-02', 4600, 20)
, (53, 2, 4, '2007-03-02', '2007-03-02', 6700, 40)
, (54, 3, 7, '2007-03-02', '2007-03-02', 2800, 25)
, (55, 5, 7, '2007-03-02', '2007-03-02', 4000, 55)
, (56, 7, 8, '2007-03-02', '2007-03-02', 3500, 75)
, (57, 4, 9, '2007-03-02', '2007-03-02', 11000, 30)
, (58, 9, 9, '2007-03-02', '2007-03-02', 2000, 10)
;""")

In [6]:
conn.execute("""USE dw;""")
conn.execute("""TRUNCATE dw.customer_stg;""")


In [7]:
df = pd.read_csv('assets/customer.csv')
df.head()

,customer_number,customer_name,customer_street_address,customer_zip_code,customer_city,customer_state
0,21,Really Large Customers,7500 Louise Dr.,17050,Mechanicsburg,PA
1,22,Small Stores,2500 Woodland St.,17055,Pittsburgh,PA
2,23,Medium Retailers,1111 Ritter Rd.,17055,Pittsburgh,PA
3,24,Good Companies,9500 Scott St.,17050,Mechanicsburg,PA
4,25,Wonderful Shops,3333 Rossmoyne Rd.,17050,Mechanicsburg,PA


In [8]:
conn.execute("USE dw;")
conn.batch_insert('customer_stg', df.columns.tolist(), df.values.tolist())

In [9]:
conn.execute("""USE dw;""")
conn.execute("""TRUNCATE table dw.customer_dim;""")
conn.execute("""
INSERT INTO customer_dim
SELECT
  NULL	
, b.customer_number
, b.customer_name
, b.customer_street_address
, b.customer_zip_code
, b.customer_city
, b.customer_state
, b.shipping_address
, b.shipping_zip_code
, b.shipping_city
, b.shipping_state
, CURRENT_DATE
, '9999-12-31'
FROM 
 customer_stg b;""")
conn.execute("""
UPDATE 
  customer_dim a
, customer_stg b
SET
  a.expiry_date = SUBDATE(CURRENT_DATE, 1)
WHERE
  a.customer_number = b.customer_number
AND expiry_date = '9999-12-31'
; """)


In [10]:
conn.execute("""TRUNCATE product_stg;""")

In [11]:
df = pd.read_csv('assets/product.csv')
df.head()

,product_code,product_name,product_category
0,1,Hard Disk Drive,Storage
1,2,Pen Drive,Storage
2,3,Flat Panel,Monitor
3,4,Keyboard,Peripheral
4,5,Mouse,Peripheral


In [12]:

conn.execute("USE dw;")
conn.batch_insert('product_stg', df.columns.tolist(), df.values.tolist())

In [13]:
conn.execute("""USE dw;""")
conn.execute("""
UPDATE dw.sales_order_fact a, source.sales_order b
SET a.order_quantity = b.order_quantity
WHERE a.customer_sk = b.customer_number
;""")


In [14]:
conn.execute("""USE source;""")
conn.execute("""INSERT INTO sales_order VALUES
  (59, 1, 1, '2007-03-02', '2007-03-02', 1000, 10)
, (60, 2, 2, '2007-03-02', '2007-03-02', 2000, 20)
, (61, 3, 3, '2007-03-02', '2007-03-02', 4000, 40)
, (62, 4, 4, '2007-03-02', '2007-03-02', 6000, 60) 
, (63, 5, 1, '2007-03-02', '2007-03-02', 2500, 25)
, (64, 6, 2, '2007-03-02', '2007-03-02', 5000, 50)
, (65, 7, 3, '2007-03-02', '2007-03-02', 7500, 75) 
, (66, 8, 4, '2007-03-02', '2007-03-02', 2300, 100)
, (67, 9, 1, '2007-03-02', '2007-03-02', 3000, 30)
, (68, 10, 1, '2007-03-02', '2007-03-02', 4100, 35)
, (69, 10, 1, '2007-03-02', '2007-03-02', 3500, 25)
, (70, 8, 1, '2007-03-02', '2007-03-02', 4000, 10);""")
conn.execute("""USE dw;""")
conn.execute("""INSERT INTO order_dim 
SELECT
  NULL
, order_number
, order_date
, '9999-12-31'
FROM
source.sales_order
WHERE order_date = '2007-03-02';""")
conn.execute("""INSERT INTO sales_order_fact
SELECT
  order_sk
, customer_sk
, product_sk
, e.date_sk
, a.order_amount
, a.order_quantity
FROM
  source.sales_order a
, dw.order_dim b
, dw.customer_dim c
, dw.product_dim d
, dw.date_dim e
WHERE
    a.order_number = b.order_number
AND a.customer_number = c.customer_number
AND a.product_code = d.product_code
AND a.order_date = e.date;""")

In [15]:
result = conn.fetch("""SELECT * FROM sales_order_fact""")
pd.DataFrame(result)

,0,1,2,3,4,5
0,87,7,9,1,7400.00,75.0
1,86,3,10,1,7800.00,40.0
2,85,10,10,1,15500.00,10.0
3,84,6,3,1,2600.00,50.0
4,83,6,8,1,1500.00,50.0
...,...,...,...,...,...,...
95260,5,5,2,1095,6000.00,25.0
95261,4,4,1,1095,4000.00,60.0
95262,3,3,3,1095,4000.00,40.0
95263,2,2,8,1095,1000.00,20.0


In [16]:
result = conn.fetch("""SELECT * FROM customer_dim""")
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,21,Really Large Customers,7500 Louise Dr.,17050,Mechanicsburg,PA,None,None,None,None,2023-08-19,2023-08-18
1,2,22,Small Stores,2500 Woodland St.,17055,Pittsburgh,PA,None,None,None,None,2023-08-19,2023-08-18
2,3,23,Medium Retailers,1111 Ritter Rd.,17055,Pittsburgh,PA,None,None,None,None,2023-08-19,2023-08-18
3,4,24,Good Companies,9500 Scott St.,17050,Mechanicsburg,PA,None,None,None,None,2023-08-19,2023-08-18
4,5,25,Wonderful Shops,3333 Rossmoyne Rd.,17050,Mechanicsburg,PA,None,None,None,None,2023-08-19,2023-08-18
5,6,26,Loyal Clients,7070 Ritter Rd.,17055,Pittsburgh,PA,None,None,None,None,2023-08-19,2023-08-18
6,7,27,Distinguished Partners,990 Scott St.,17050,Mechanicsburg,PA,None,None,None,None,2023-08-19,2023-08-18
7,8,28,Distro Inc,1290 Scott St.,17050,Pittsburgh,PA,None,None,None,None,2023-08-19,2023-08-18
8,9,29,ACME Inc,90 Savoy St.,15001,California,CA,None,None,None,None,2023-08-19,2023-08-18
9,10,30,Docs Company,1340 Doc Av.,29350,San Francisco,CA,None,None,None,None,2023-08-19,2023-08-18


Cria partição

In [17]:
conn.execute("""USE dw;""")
conn.execute("""ALTER TABLE SALES_ORDER_FACT PARTITION BY RANGE( ORDER_DATE_SK ) ( 
PARTITION P0 VALUES LESS THAN (479), 
PARTITION P1 VALUES LESS THAN (988), 
PARTITION P2 VALUES LESS THAN (1188), 
PARTITION P3 VALUES LESS THAN (1288));""")

In [18]:
result = conn.fetch("SELECT * FROM information_schema.partitions WHERE TABLE_SCHEMA='DW' AND TABLE_NAME = 'SALES_ORDER_FACT';")
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,def,dw,SALES_ORDER_FACT,P3,None,4,None,RANGE,None,"[96, 79, 82, 68, 69, 82, 95, 68, 65, 84, 69, 9...",...,0,0,0,2023-08-19 15:59:54,None,None,None,[],default,None
1,def,dw,SALES_ORDER_FACT,P2,None,3,None,RANGE,None,"[96, 79, 82, 68, 69, 82, 95, 68, 65, 84, 69, 9...",...,0,0,0,2023-08-19 15:59:54,None,None,None,[],default,None
2,def,dw,SALES_ORDER_FACT,P1,None,2,None,RANGE,None,"[96, 79, 82, 68, 69, 82, 95, 68, 65, 84, 69, 9...",...,0,0,0,2023-08-19 15:59:54,None,None,None,[],default,None
3,def,dw,SALES_ORDER_FACT,P0,None,1,None,RANGE,None,"[96, 79, 82, 68, 69, 82, 95, 68, 65, 84, 69, 9...",...,0,0,0,2023-08-19 15:59:54,None,None,None,[],default,None


In [19]:
result = conn.fetch("select count(*) from SALES_ORDER_FACT partition (p2);")
pd.DataFrame(result)

,0
0,9396
